In [1]:
import matplotlib.pyplot as plt
import numpy as np
import json

In [3]:
def extract_keypoints_from_a_json(origin_vid_dir):
    with open(origin_vid_dir, "r") as f:
        origin_keypoints = json.load(f)

    origin_keypoints_anno = origin_keypoints['annotations']
    origin_tmp_list = []
    for frame_data in origin_keypoints_anno:
        if frame_data:
            keypoints_list = [frame_data[key] for key in frame_data if frame_data[key]]
            if keypoints_list:
                origin_tmp_list.extend(keypoints_list)

    # # Nonetype error 발생하는 듯
    # if not origin_tmp_list:
    #     return None

    # reshape x
    origin_anchor_keypoints = np.array(origin_tmp_list)[:,:,[0,1]]
    return origin_anchor_keypoints

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
from PIL import Image

data = extract_keypoints_from_a_json('dataset/a001_kp/S001C001P001R001A001_rgb_kp.json')
points = data.reshape(-1,2)
x_points = points[:,[0]]
y_points = points[:,[1]]

h = 1080
w = 1920
# w = 1920*(1080/1920)

H = np.tile(np.arange(h)[:, None], w)
W = np.tile(np.arange(w)[:, None], h).T
# print(H)
# print(W)
# print(W.shape)
std = 5

# org_image = Image.open('frame0.jpg')
# resized_image = org_image.resize((int(w), h))
frame_list = []

fig = plt.figure(frameon=False)
ax = fig.add_axes([0,0,1,1])
ax.axis('off')

for i in range(len(x_points)):
    x = x_points[i]
    y = y_points[i]
    # y = y_points[i]*(1080/1920)
    x_gauss = 1/(np.sqrt(2*np.pi)*std) * np.exp(-0.5*((H-x)/std)**2) 
    y_gauss = 1/(np.sqrt(2*np.pi)*std) * np.exp(-0.5*((W-y)/std)**2)
    frame = (x_gauss * y_gauss)

    frame_list.append(frame)

cnt = 1
for i in range(len(frame_list)):
    if i%17 == 0:
        tmp_frame = frame_list[i]
    else:
        tmp_frame += frame_list[i]
        if cnt % 17 == 0:
            ax.imshow(tmp_frame, aspect='auto')
            fig.savefig(f'heatmap_sum/A001_{int(cnt/17)}.jpg', bbox_inches='tight', pad_inches = 0)
    cnt += 1
    
# print(tmp_frame)
# print(tmp_frame.shape)

# plt.imshow(tmp_frame)
    # # plt.imshow(org_image)
    # # plt.imshow(frame, alpha = 0.6, cmap = 'viridis')
    # fig = plt.figure(frameon=False)
    # ax = fig.add_axes([0,0,1,1])
    # ax.axis('off')
    # ax.imshow(frame, aspect='auto')
    # # plt.savefig(f'overlap/A001_{i}.png')
    # fig.savefig(f'heatmap_sum/A001_{i}.jpg', bbox_inches='tight', pad_inches = 0)
    # print(i)
    # # overlap = frame * 0.4 + resized_image

    # # overlap_pil = Image.fromarray(overlap)
    # # overlap_pil.show()

    # # overlap_pil.save(f'overlap/frame{i}.png','png')

    # # ax = sns.heatmap(overlap)
    # # plt.show()
